<a href="https://colab.research.google.com/github/matsudsatsuki/AtCoder/blob/main/Presentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>





---


# 研究力調査

---



##  概要
- Scivalから抽出した2012-2021年の論文データについて、3年単位を1年ごとにずらし(2012-2014、2013-2015...)、研究力の推移を機関ごと、分野ごとに集計、分析し研究力の動向をとりまとめる。

- 条件を満たす研究者の推移や分野の割合を他大学と比較する

## 目的
- 条件を満たす研究者*の推移や分野の割合を他大学と比較することで、東工大の研究力の推移や分野ごとの特徴を可視化する

- 東工大と医科歯科大が合併した際の研究力を可視化する

> *条件：3年で文献数40本以上またはFWCI Top10%に入る文献が同期間で8本以上を満たす

> *FWCI：論文の被引用数を、その論文と同じ出版年・分野・文献種の論文の世界平均（基準値）化した指標.

## 調査対象
- 2012年~2021年の東工大と海外のベンチマーク大学(MIT,Imperial College London),日本の主要大学(東大、京大、阪大、名古屋大、東北大、東京医科歯科大)を比較











### 前処理

In [105]:
!pip install japanize-matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [106]:
!pip install plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [107]:
#各種import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_profiling as pp
from pandas import plotting
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, precision_score, recall_score 
import warnings
import lightgbm
import os
import glob
from glob import glob
import re
from collections import defaultdict
import japanize_matplotlib
from pandas.core.indexes.base import zip_longest
import requests
from bs4 import BeautifulSoup
import time
import plotly.express as px
import plotly.graph_objects as go
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
%matplotlib inline

In [108]:
path = '/content/drive/MyDrive/SciVal/'

In [109]:
titech = pd.read_csv(path+'titech_res_list.csv')
UT = pd.read_csv(path+'UT_res_list.csv')
kyoto = pd.read_csv(path+'kyoto_res_list.csv')
osaka = pd.read_csv(path+'osaka_res_list.csv')
nagoya = pd.read_csv(path+'nagoya_res_list.csv')
medical = pd.read_csv(path+'medical_res_list.csv')
MIT = pd.read_csv(path+'MIT_res_list.csv')
Imperial = pd.read_csv(path+'Imperial_res_list.csv')

In [110]:
pd.merge(nagoya,Imperial, how='inner')

,Unnamed: 0,Scopus Author Ids


In [111]:
titech = titech[['Scopus Author Ids']]
UT = UT[['Scopus Author Ids']]
kyoto = kyoto[['Scopus Author Ids']]
osaka = osaka[['Scopus Author Ids']]
nagoya = nagoya[['Scopus Author Ids']]
medical = medical[['Scopus Author Ids']]
MIT = MIT[['Scopus Author Ids']]
Imperial = Imperial[['Scopus Author Ids']]

In [112]:
univ_list = ['東工大','東大','京大','阪大','名古屋大','医科歯科大','東北大','MIT','Imperial']
univ_ = ['titech','UT','kyoto','osaka','nagoya','medical','tohoku','MIT','Imperial']
all_res = pd.DataFrame()
for univ,name in zip(univ_list,univ_):
  start_year = 2012
  end_year = 2014
  allFiles = glob(path+f'{univ}20*.csv')   
  #frame = pd.DataFrame() 
  for file_ in allFiles:
    df = pd.read_csv(file_,encoding = 'cp932')
    df['univ_name'] = name
    df['start_year'] = start_year
    df['end_year'] = end_year
    #print(df)
    all_res = all_res.append(df)
    start_year += 1
    end_year += 1


<ipython-input-112-c58f5e65187c>:15: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-112-c58f5e65187c>:15: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-112-c58f5e65187c>:15: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-112-c58f5e65187c>:15: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-112-c58f5e65187c>:15: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-112-c58f5e65187c>:15: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future versi

In [113]:
from pandas.core.ops import flex_comp_method_FRAME
univ = ['MIT','Imperial','kyoto','titech','tohoku','nagoya','medical','osaka','UT']

files = glob(path+'*_researcher_tag.csv')
tag_df = pd.DataFrame()
for file_,univ_name in zip(files,univ):
  print(file_)
  df1 = pd.read_csv(file_)
  df1['univ_name'] = univ_name
  tag_df = tag_df.append(df1)

/content/drive/MyDrive/SciVal/kyoto_researcher_tag.csv
/content/drive/MyDrive/SciVal/nagoya_researcher_tag.csv
/content/drive/MyDrive/SciVal/medical_researcher_tag.csv
/content/drive/MyDrive/SciVal/UT_researcher_tag.csv
/content/drive/MyDrive/SciVal/Imperial_researcher_tag.csv
/content/drive/MyDrive/SciVal/osaka_researcher_tag.csv
/content/drive/MyDrive/SciVal/tohoku_researcher_tag.csv
/content/drive/MyDrive/SciVal/titech_researcher_tag.csv
/content/drive/MyDrive/SciVal/MIT_researcher_tag.csv


<ipython-input-113-7f8d4bc0e98c>:10: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-113-7f8d4bc0e98c>:10: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-113-7f8d4bc0e98c>:10: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-113-7f8d4bc0e98c>:10: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-113-7f8d4bc0e98c>:10: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-113-7f8d4bc0e98c>:10: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future versi

In [114]:
#日本の大学の2018~2020年までの研究者数（教授、准教授、助教）[2018,2019,2020]

titech_all = [1030,1041,1031]
UT_all = [3531,3544,3606]
kyoto_all = [2529,2482,2507]
osaka_all = []
nagoya_all = [2071,2047,2037]
tohoku_all = []
print(sum(titech_all))


3102


In [115]:
#日本の大学の分野別の研究費[2018,2019,2020]億円

titech_budget = [134,123,121]
UT_budget = [584,548,579]
kyoto_budget = [368,345,370]
osaka_budget = [275,292,283]
nagoya_budget = [185,186,195]
tohoku_budget = [298,257,320]

In [116]:
tag_df = tag_df.drop('Unnamed: 0',axis=1)

In [117]:
tag_df['univ_name'].value_counts()
#tag_dfの大学ごとの数値が間違っている

UT          1244
tohoku      1054
titech       806
medical      475
MIT          462
nagoya       398
Imperial     285
osaka        183
kyoto         72
Name: univ_name, dtype: int64

In [118]:
a = all_res[all_res['univ_name'] == 'titech']
a['Scopus Author Ids'].drop_duplicates().shape[0]
#人数確認

183

In [119]:
all_res.head()

,Unnamed: 0,Scopus Author Ids,univ_name,start_year,end_year
0,2,6602154752,titech,2012,2014
1,3,36040908600,titech,2012,2014
2,9,7202652119,titech,2012,2014
3,11,7102848390,titech,2012,2014
4,14,7102758574,titech,2012,2014


In [120]:
all_res['start_year'].value_counts()

2013    2678
2012    2602
2019    2531
2018    2497
2016    2455
2017    2449
2015    2421
2014    2419
Name: start_year, dtype: int64

In [121]:
all_res['start_year'].value_counts()

2013    2678
2012    2602
2019    2531
2018    2497
2016    2455
2017    2449
2015    2421
2014    2419
Name: start_year, dtype: int64

In [122]:
titech_list = [x for x in range(8)]
UT_list = [x for x in range(8)]
kyoto_list = [x for x in range(8)]
osaka_list = [x for x in range(8)]
nagoya_list = [x for x in range(8)]
medical_list = [x for x in range(8)]
MIT_list = [x for x in range(8)]
tohoku_list = [x for x in range(8)]
Imperial_list = [x for x in range(8)]
#titech_med_list = [x for x in range(8)]
univ_numlist = [titech_list,UT_list,kyoto_list,osaka_list,nagoya_list,medical_list,tohoku_list,MIT_list,Imperial_list]
#titech_list[0] = 2
#univ_list[0][0] = 1
#titech_list


In [123]:
for i,name in enumerate(univ_):
  s = 2012
  for j in range(8):
    #print(s+j)
    df = all_res[(all_res['univ_name'] == name) & (all_res['start_year'] == s+j)]
    univ_numlist[i][j] = df['Scopus Author Ids'].drop_duplicates().shape[0]


In [124]:
univ_numlist

[[81, 76, 78, 64, 67, 59, 69, 62],
 [356, 383, 372, 411, 432, 453, 434, 414],
 [218, 222, 223, 224, 219, 218, 232, 223],
 [233, 223, 200, 183, 184, 187, 181, 180],
 [132, 138, 138, 134, 145, 162, 166, 163],
 [31, 29, 30, 27, 35, 39, 41, 57],
 [244, 241, 204, 183, 190, 174, 198, 229],
 [714, 722, 751, 718, 681, 666, 656, 644],
 [593, 644, 423, 477, 502, 491, 520, 559]]

In [125]:
univ = pd.DataFrame(columns=['大学名','条件を満たす研究者の数','year'])
year_list = ['2012~2014','2013~2015','2014~2016','2015~2017','2016~2018','2017~2019','2018~2020','2019~2021']
univ_df = pd.DataFrame()
for i,univ_name in enumerate(univ_list):
  j = 0
  for num,year in zip(univ_numlist[i],year_list):
    #print(year)
    #print(num)
    univ.at[j,'大学名'] = univ_name
    univ.at[j,'条件を満たす研究者の数'] = num
    univ.at[j,'year'] = year
    j += 1
  univ_df = pd.concat([univ_df,univ])
  #print(univ_name)

In [126]:
univ_df.head()

,大学名,条件を満たす研究者の数,year
0,東工大,81,2012~2014
1,東工大,76,2013~2015
2,東工大,78,2014~2016
3,東工大,64,2015~2017
4,東工大,67,2016~2018


In [127]:
tit = univ_df[univ_df['大学名'] == '東工大']
med = univ_df[univ_df['大学名'] == '医科歯科大']
tit_med = pd.DataFrame(columns=['大学名','条件を満たす研究者の数','year'])
i = 0
tit_med['year'] = year_list
for t,m in zip(tit['条件を満たす研究者の数'],med['条件を満たす研究者の数']):
  tit_med.at[i,'大学名'] = '東工大＋医科歯科大'
  tit_med.at[i,'条件を満たす研究者の数'] = t+m
  i += 1


In [128]:
tit_med

,大学名,条件を満たす研究者の数,year
0,東工大＋医科歯科大,112,2012~2014
1,東工大＋医科歯科大,105,2013~2015
2,東工大＋医科歯科大,108,2014~2016
3,東工大＋医科歯科大,91,2015~2017
4,東工大＋医科歯科大,102,2016~2018
5,東工大＋医科歯科大,98,2017~2019
6,東工大＋医科歯科大,110,2018~2020
7,東工大＋医科歯科大,119,2019~2021


In [129]:
univ_df = pd.concat([univ_df,tit_med])

In [130]:
univ_df.head()

,大学名,条件を満たす研究者の数,year
0,東工大,81,2012~2014
1,東工大,76,2013~2015
2,東工大,78,2014~2016
3,東工大,64,2015~2017
4,東工大,67,2016~2018


In [131]:
all_res['univ_name'].value_counts()

MIT         5552
Imperial    4209
UT          3255
kyoto       1779
tohoku      1663
osaka       1571
nagoya      1178
titech       556
medical      289
Name: univ_name, dtype: int64

In [132]:
all_res.shape[0]

20052

In [133]:
all_res.head()

,Unnamed: 0,Scopus Author Ids,univ_name,start_year,end_year
0,2,6602154752,titech,2012,2014
1,3,36040908600,titech,2012,2014
2,9,7202652119,titech,2012,2014
3,11,7102848390,titech,2012,2014
4,14,7102758574,titech,2012,2014


In [134]:
all_year_res = all_res.drop_duplicates(subset=['Scopus Author Ids','univ_name'])

In [135]:
#全期間で条件を満たす研究者の数
all_year_res['univ_name'].value_counts()

MIT         1676
Imperial    1291
UT           934
kyoto        525
tohoku       475
osaka        456
nagoya       320
titech       183
medical       86
Name: univ_name, dtype: int64

In [136]:
tag_df.head()

,Scopus Author Ids,All Science Journal Classification (ASJC) code,tag,univ_name
0,7103161459,27,"General Pharmacology, Toxicology and Pharmaceu...",MIT
1,7006377392,27,General Medicine（一般医学）,MIT
2,7402420635,27,General Medicine（一般医学）,MIT
3,7102140072,27,General Physics and Astronomy（一般物理・天文学）,MIT
4,7202185709,27,General Chemistry（一般化学）,MIT


In [137]:
tag_df.shape[0]

4979

In [138]:
all_res.head()

,Unnamed: 0,Scopus Author Ids,univ_name,start_year,end_year
0,2,6602154752,titech,2012,2014
1,3,36040908600,titech,2012,2014
2,9,7202652119,titech,2012,2014
3,11,7102848390,titech,2012,2014
4,14,7102758574,titech,2012,2014


In [139]:
tag_df.head()

,Scopus Author Ids,All Science Journal Classification (ASJC) code,tag,univ_name
0,7103161459,27,"General Pharmacology, Toxicology and Pharmaceu...",MIT
1,7006377392,27,General Medicine（一般医学）,MIT
2,7402420635,27,General Medicine（一般医学）,MIT
3,7102140072,27,General Physics and Astronomy（一般物理・天文学）,MIT
4,7202185709,27,General Chemistry（一般化学）,MIT


In [140]:
tag_all = pd.merge(all_res,tag_df,how='inner',on='Scopus Author Ids')

In [141]:
tag_all.shape[0]

17897

In [142]:
tag_all.query('univ_name_x == "titech"').shape[0]

618

In [143]:
a = tag_all[tag_all['univ_name_x'] == 'titech']

In [144]:
a['Scopus Author Ids'].nunique()

183

In [145]:
df_3 = tag_all[(tag_all['start_year'] == 2012) | (tag_all['start_year'] == 2015) | (tag_all['start_year'] == 2018) | (tag_all['start_year'] == 2022)]

In [146]:
df_3['start_year'].value_counts()

2012    2264
2018    2246
2015    2179
Name: start_year, dtype: int64

In [147]:
df_3 = df_3.rename(columns={'univ_name_x':'univ_name'})

In [148]:
df_3['researcher_num'] = 1

In [149]:
a = df_3[df_3['univ_name'] == 'titech']

In [150]:
all_y_res = tag_all.drop_duplicates(subset=['Scopus Author Ids','univ_name_x'])

In [151]:
all_y_res['univ_name_x'].value_counts()

MIT         1246
Imperial    1054
UT           815
tohoku       475
kyoto        469
osaka        405
nagoya       285
titech       183
medical       74
Name: univ_name_x, dtype: int64

# ①各大学における条件を満たす研究者の人数と医学分野の割合

## 集計

In [48]:
all_y_res['researcher_num'] = 1
all_y_res['flag'] = '医学以外'

<ipython-input-48-9b5296ae57cb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_y_res['researcher_num'] = 1
<ipython-input-48-9b5296ae57cb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_y_res['flag'] = '医学以外'


In [49]:
all_y_res.loc[all_y_res['tag'] == 'General Medicine（一般医学）','flag'] = '医学'

In [50]:
df_japan = df_3[(df_3['univ_name'] != 'MIT')|(df_3['univ_name'] != 'Imperial')]

In [51]:
df_japan['tag'].value_counts()

General Medicine（一般医学）                                               1802
General Physics and Astronomy（一般物理・天文学）                              1128
General Engineering（一般工学）                                             805
General Materials Science（一般材料科学）                                     468
Biochemistry, Genetics and Molecular Biology（一般生化学・遺伝学・分子生物学）         461
General Chemistry（一般化学）                                               407
General Computer Science（一般コンピューター科学）                                 405
General Earth and Planetary Sciences（一般地球惑星科学）                        216
Multidisciplinary（学際分野）                                               183
General Mathematics（一般数学）                                             111
General Neuroscience（一般神経科学）                                          106
General Environmental Science（一般環境科学）                                 101
General Energy（一般エネルギー）                                                73
General Chemical Engineering（一般化学工学）  

In [52]:
ss = defaultdict(int)
for v,k in zip(all_y_res['univ_name_x'].value_counts().values,all_y_res['univ_name_x'].value_counts().keys()):
  ss[k] = all_y_res[all_y_res['flag'] == '医学']['univ_name_x'].value_counts()[k]/v
#ss
med = all_y_res[all_y_res['univ_name_x'] == 'titech'].shape[0] + all_y_res[all_y_res['univ_name_x'] == 'medical'].shape[0]
num_medical = all_y_res[(all_y_res['flag'] == '医学') & (all_y_res['univ_name_x'] == 'titech')].shape[0] + all_y_res[(all_y_res['flag'] == '医学') & (all_y_res['univ_name_x'] == 'medical')].shape[0]
num_notmedical = all_y_res[(all_y_res['flag'] != '医学') & (all_y_res['univ_name_x'] == 'titech')].shape[0] + all_y_res[(all_y_res['flag'] != '医学') & (all_y_res['univ_name_x'] == 'medical')].shape[0]

u = pd.DataFrame(ss.values(), index=ss.keys())
u = u.reset_index()
u.rename(columns={'index':'univ_name_x',0:'医学割合'},inplace=True)
u = u.sort_values(by='univ_name_x')
b = pd.DataFrame([['titech+medical',num_medical/med]],columns=['univ_name_x','医学割合'])
u = u.append(b)

<ipython-input-52-575f42fcb3cd>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  u = u.append(b)


In [53]:
aa = pd.DataFrame()
aa['univ_name_x'] = all_y_res[all_y_res['flag'] != '医学']['univ_name_x'].value_counts().keys()
aa['医学以外'] = all_y_res[all_y_res['flag'] != '医学']['univ_name_x'].value_counts().values
aa = aa.sort_values(by='univ_name_x')
c = pd.DataFrame([['titech+medical',num_notmedical]],columns=['univ_name_x','医学以外'])
aa = aa.append(c)


<ipython-input-53-3bcdcf5bd80d>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  aa = aa.append(c)


In [54]:
bb = pd.DataFrame()
bb['univ_name_x'] = all_y_res[all_y_res['flag'] == '医学']['univ_name_x'].value_counts().keys()
bb['医学数'] = all_y_res[all_y_res['flag'] == '医学']['univ_name_x'].value_counts().values
bb = bb.sort_values(by='univ_name_x')
d = pd.DataFrame([['titech+medical',num_medical]],columns=['univ_name_x','医学数'])
bb = bb.append(d)

<ipython-input-54-58611449e134>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bb = bb.append(d)


In [55]:
d_order = {'東工大': 0, '東工大+医科歯科大':1,'MIT': 2,'Imperial College London': 3, '東大': 4,'京大':5,'阪大':6,'東北大':7,'名古屋大':8, '東京医科歯科大':9}

aa.loc[aa['univ_name_x'] == 'titech','univ_name_x'] = '東工大'
aa.loc[aa['univ_name_x'] == 'Imperial','univ_name_x'] = 'Imperial College London'
aa.loc[aa['univ_name_x'] == 'UT','univ_name_x'] = '東大'
aa.loc[aa['univ_name_x'] == 'kyoto','univ_name_x'] = '京大'
aa.loc[aa['univ_name_x'] == 'osaka','univ_name_x'] = '阪大'
aa.loc[aa['univ_name_x'] == 'tohoku','univ_name_x'] = '東北大'
aa.loc[aa['univ_name_x'] == 'nagoya','univ_name_x'] = '名古屋大'
aa.loc[aa['univ_name_x'] == 'medical','univ_name_x'] = '東京医科歯科大'
aa.loc[aa['univ_name_x'] == 'titech+medical','univ_name_x'] = '東工大+医科歯科大'

bb.loc[bb['univ_name_x'] == 'titech','univ_name_x'] = '東工大'
bb.loc[bb['univ_name_x'] == 'Imperial','univ_name_x'] = 'Imperial College London'
bb.loc[bb['univ_name_x'] == 'UT','univ_name_x'] = '東大'
bb.loc[bb['univ_name_x'] == 'kyoto','univ_name_x'] = '京大'
bb.loc[bb['univ_name_x'] == 'osaka','univ_name_x'] = '阪大'
bb.loc[bb['univ_name_x'] == 'tohoku','univ_name_x'] = '東北大'
bb.loc[bb['univ_name_x'] == 'nagoya','univ_name_x'] = '名古屋大'
bb.loc[bb['univ_name_x'] == 'medical','univ_name_x'] = '東京医科歯科大'
bb.loc[bb['univ_name_x'] == 'titech+medical','univ_name_x'] = '東工大+医科歯科大'

u.loc[u['univ_name_x'] == 'titech','univ_name_x'] = '東工大'
u.loc[u['univ_name_x'] == 'Imperial','univ_name_x'] = 'Imperial College London'
u.loc[u['univ_name_x'] == 'UT','univ_name_x'] = '東大'
u.loc[u['univ_name_x'] == 'kyoto','univ_name_x'] = '京大'
u.loc[u['univ_name_x'] == 'osaka','univ_name_x'] = '阪大'
u.loc[u['univ_name_x'] == 'tohoku','univ_name_x'] = '東北大'
u.loc[u['univ_name_x'] == 'nagoya','univ_name_x'] = '名古屋大'
u.loc[u['univ_name_x'] == 'medical','univ_name_x'] = '東京医科歯科大'
u.loc[u['univ_name_x'] == 'titech+medical','univ_name_x'] = '東工大+医科歯科大'

aa['order'] = aa['univ_name_x'].map(d_order)
aa = aa.sort_values('order')
bb['order'] = bb['univ_name_x'].map(d_order)
bb = bb.sort_values('order')
u['order'] = u['univ_name_x'].map(d_order)
u = u.sort_values('order')

In [56]:
#参考値としての最新の大学の研究者数
titech = 1033
nagoya = 2296
tohoku = 3174
MIT = 1069
kyoto = 3489
Imperial = 1358
osaka = 3306
UT = 3945
medical = 853
titech_med = titech+medical
res_sum = [titech,UT,kyoto,osaka,nagoya,tohoku,medical,MIT,Imperial,titech_med]
label_name = ['titech','UT','kyoto','osaka','nagoya','tohoku','medical','MIT','Imperial','titech+medical']
cc = pd.DataFrame()
cc['univ_name_x'] = label_name
cc['研究者の数'] = res_sum
cc.loc[cc['univ_name_x'] == 'titech','univ_name_x'] = '東工大'
cc.loc[cc['univ_name_x'] == 'Imperial','univ_name_x'] = 'Imperial College London'
cc.loc[cc['univ_name_x'] == 'UT','univ_name_x'] = '東大'
cc.loc[cc['univ_name_x'] == 'kyoto','univ_name_x'] = '京大'
cc.loc[cc['univ_name_x'] == 'osaka','univ_name_x'] = '阪大'
cc.loc[cc['univ_name_x'] == 'tohoku','univ_name_x'] = '東北大'
cc.loc[cc['univ_name_x'] == 'nagoya','univ_name_x'] = '名古屋大'
cc.loc[cc['univ_name_x'] == 'medical','univ_name_x'] = '東京医科歯科大'
cc.loc[cc['univ_name_x'] == 'titech+medical','univ_name_x'] = '東工大+医科歯科大'

cc['order'] = cc['univ_name_x'].map(d_order)
cc = cc.sort_values('order')
cc = cc.rename(columns={'univ_name_x': '大学名'})

In [57]:
cc

,大学名,研究者の数,order
0,東工大,1033,0
9,東工大+医科歯科大,1886,1
7,MIT,1069,2
8,Imperial College London,1358,3
1,東大,3945,4
2,京大,3489,5
3,阪大,3306,6
5,東北大,3174,7
4,名古屋大,2296,8
6,東京医科歯科大,853,9


In [58]:
# プロットの作成
fig = px.bar(cc, x='大学名', y='研究者の数')

# x軸のラベルの回転
fig.update_layout(xaxis_tickangle=-45)

fig.update_traces(
    selector= 'bar',
    hovertemplate='%{x}: %{y:1f}',
    texttemplate='%{y:1f}',
    textposition='outside'
)
# プロットの表示

fig.update_layout(
    title = '2022年の大学ごとの研究者数(参考値)'
)
fig.show()

In [59]:
#ここに二つグラフを書く(絶対数のグラフと、分母のグラフ)
from plotly.subplots import make_subplots
# Y軸に2軸目が存在するサブプロット
two_yaxis_fig = make_subplots(specs=[[{"secondary_y": True}]]) 
two_yaxis_fig.add_trace(go.Bar(x = aa['univ_name_x'],
                               y = aa['医学以外'],
                               name='医学以外'))

two_yaxis_fig.add_trace(go.Bar(x = bb['univ_name_x'],
                               y = bb['医学数'],
                               name='医学'))

two_yaxis_fig.add_trace(go.Scatter(x = u['univ_name_x'],
                                   y = u['医学割合']*100,
                                   name="医学割合"),secondary_y=True)

two_yaxis_fig.update_yaxes(title={"text": "条件を満たす研究者の数"}, showgrid=False)

two_yaxis_fig.update_yaxes(
    secondary_y=True,  # 2軸目の設定
    title={"text": "医学割合"},  # 軸ラベルの指定
    showgrid=False  # 補助線を非表示
)

two_yaxis_fig.update_traces(
    mode='markers+text',
    marker_symbol='arrow-bar-up',  # マーカーの形状を四角形に
    marker_size=10,  # マーカーのサイズを30に
    textposition='top center',
    texttemplate = '%{y:.2f}%',
    selector= 'scatter'  # selectorでどのプロットに対して更新するか選択可能
)
two_yaxis_fig.update_traces(
    selector= 'bar',
    hovertemplate='%{x}: %{y:1f}',
    texttemplate='%{y:1f}',
    textposition='outside'
)


two_yaxis_fig.update_layout(
    title = '2012~2021年の条件を満たす研究者の数と医学割合',
    barmode='stack',

)


y_axis = two_yaxis_fig
two_yaxis_fig.show()

## 可視化結果

- 東工大の条件を満たす研究者の絶対数はベンチマーク大学や日本の主要大学と比較すると低くなっている

- 大学の規模感から考えると、東工大の条件を満たす研究者の割合は比較的高い（※下図参照）
- 医学を除いた場合東工大の条件を満たす研究者の数は名古屋大を上回っている

- 東工大と医科歯科大学が以前から合併していたと仮定した場合、条件を満たす研究者の数は名古屋大学と同等になる(東工大+医科歯科大：257人, 名古屋大：285人)

- 日本の大学の条件を満たす研究者の医学割合は20~30%ほどを占めている。
それに対して海外のベンチマーク大学は医学の割合が高低はっきりしている



In [60]:
two_yaxis_fig.show()
fig.show()

# ②分野別の条件を満たす研究者割合

### 分野についてはASJC（All Science Journal Classification）に従って分類した。理学と工学については本学の基準による

- 理学


> Mathematics (数学),
Physics and Astronomy (物理・天文学),
Chemistry (化学),
Earth and Planetary Sciences (地球惑星科学),
Environmental Science (環境科学),


- 工学


> Engineering (工学),
Chemical Engineering (化学工学),
Material Science (材料科学),
Computer Science (コンピュータ科学),
Energy (エネルギー),




- 医学

> Health Sciences (ヘルスサイエンス),
Medicine (医学),
Nursing (看護学),
Veterinary (獣医学),
Dentistry (歯学),
Health Professions (保健医療),




- 社会科学


> Social Sciences (社会科学)	Arts and Humanities (人文科学),
Business, Management and Accounting (ビジネス・マネジメント・会計学),
Decision Sciences (決定科学),
Economics, Econometrics and Finance (経済学・計量経済学・財政),
Psychology (心理学),

- 生命科学

> Life Sciences (ライフサイエンス),
Agricultural and Biological Sciences (農学・生物科学),
Biochemistry, Genetics and Molecular Biology (生化学・遺伝学・分子生物学),
Immunology and Microbiology (免疫学・微生物学),
Neuroscience (神経科学),
Pharmacology, Toxicology and Pharmaceutics (薬理学・毒物学・薬剤学),

- その他
Multidisciplinary (学際分野),etc





## 集計

In [61]:
#tagを集計し直す,学際分野を除いた
tag_all['new_tag'] = 'その他'
#理学
tag_all.loc[(tag_all['tag'] == 'General Physics and Astronomy（一般物理・天文学）') | 
              (tag_all['tag'] == 'General Chemistry（一般化学）') |
              (tag_all['tag'] == 'General Mathematics（一般数学）') | 
              (tag_all['tag'] == 'General Earth and Planetary Sciences（一般地球惑星科学）') |
              (tag_all['tag'] == 'General Environmental Science（一般環境科学）'),'new_tag'] = '理学'

#工学
tag_all.loc[(tag_all['tag'] == 'General Engineering（一般工学）') | 
              (tag_all['tag'] == 'General Materials Science（一般材料科学）') | 
              (tag_all['tag'] == 'General Chemical Engineering（一般化学工学）') |
              (tag_all['tag'] == 'General Computer Science（一般コンピューター科学）')|
              (tag_all['tag'] == 'General Energy（一般エネルギー）'),'new_tag'] = '工学'

#医学
tag_all.loc[(tag_all['tag'] == 'General Medicine（一般医学）') | 
              (tag_all['tag'] == 'General Nursing（一般看護）') | 
              (tag_all['tag'] == 'General Health Professions（一般保健医療）') |
              (tag_all['tag'] == 'General Veterinary（一般獣医学）')|
              (tag_all['tag'] == 'General Dentistry（一般歯学）'),'new_tag'] = '医学'

#社会科学
tag_all.loc[(tag_all['tag'] == 'General Arts and Humanitie（一般人文科学）') | 
              (tag_all['tag'] == 'General Business, Management and Accounting（一般ビジネス・マネジメント・会計学）') | 
              (tag_all['tag'] == 'General Decision Sciences（一般決定科学）') |
              (tag_all['tag'] == 'General Economics, Econometrics and Finance（一般経済学・計量経済学・財政）')|
              (tag_all['tag'] == 'General Psychology（一般心理学）') |
              (tag_all['tag'] == 'General Social Sciences（一般社会科学）'),'new_tag'] = '社会科学'

#生命科学
tag_all.loc[(tag_all['tag'] == 'General Agricultural and Biological Sciences（一般農学・生物科学）') | 
              (tag_all['tag'] == 'Biochemistry, Genetics and Molecular Biology（一般生化学・遺伝学・分子生物学）') | 
              (tag_all['tag'] == 'General Immunology and Microbiology（一般免疫学・微生物学）') |
              (tag_all['tag'] == 'General Neuroscience（一般神経科学）')|
              (tag_all['tag'] == 'General Pharmacology, Toxicology and Pharmaceutics（一般薬理学・毒物学・薬剤学）') ,'new_tag'] = '生命科学'

a = tag_all.query('univ_name_x == "titech" or univ_name_x == "medical"')
a['univ_name_x'] = '東工大+医科歯科大'
tag_all = pd.concat([tag_all,a])

<ipython-input-61-dcc479616155>:40: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [62]:
#tagを集計し直す,学際分野を除いた
all_y_res['new_tag'] = 'その他'
#理学
all_y_res.loc[(all_y_res['tag'] == 'General Physics and Astronomy（一般物理・天文学）') | 
              (all_y_res['tag'] == 'General Chemistry（一般化学）') |
              (all_y_res['tag'] == 'General Mathematics（一般数学）') | 
              (all_y_res['tag'] == 'General Earth and Planetary Sciences（一般地球惑星科学）') |
              (all_y_res['tag'] == 'General Environmental Science（一般環境科学）'),'new_tag'] = '理学'

#工学
all_y_res.loc[(all_y_res['tag'] == 'General Engineering（一般工学）') | 
              (all_y_res['tag'] == 'General Materials Science（一般材料科学）') | 
              (all_y_res['tag'] == 'General Chemical Engineering（一般化学工学）') |
              (all_y_res['tag'] == 'General Computer Science（一般コンピューター科学）')|
              (all_y_res['tag'] == 'General Energy（一般エネルギー）'),'new_tag'] = '工学'

#医学
all_y_res.loc[(all_y_res['tag'] == 'General Medicine（一般医学）') | 
              (all_y_res['tag'] == 'General Nursing（一般看護）') | 
              (all_y_res['tag'] == 'General Health Professions（一般保健医療）') |
              (all_y_res['tag'] == 'General Veterinary（一般獣医学）')|
              (all_y_res['tag'] == 'General Dentistry（一般歯学）'),'new_tag'] = '医学'

#社会科学
all_y_res.loc[(all_y_res['tag'] == 'General Arts and Humanitie（一般人文科学）') | 
              (all_y_res['tag'] == 'General Business, Management and Accounting（一般ビジネス・マネジメント・会計学）') | 
              (all_y_res['tag'] == 'General Decision Sciences（一般決定科学）') |
              (all_y_res['tag'] == 'General Economics, Econometrics and Finance（一般経済学・計量経済学・財政）')|
              (all_y_res['tag'] == 'General Psychology（一般心理学）') |
              (all_y_res['tag'] == 'General Social Sciences（一般社会科学）'),'new_tag'] = '社会科学'

#生命科学
all_y_res.loc[(all_y_res['tag'] == 'General Agricultural and Biological Sciences（一般農学・生物科学）') | 
              (all_y_res['tag'] == 'Biochemistry, Genetics and Molecular Biology（一般生化学・遺伝学・分子生物学）') | 
              (all_y_res['tag'] == 'General Immunology and Microbiology（一般免疫学・微生物学）') |
              (all_y_res['tag'] == 'General Neuroscience（一般神経科学）')|
              (all_y_res['tag'] == 'General Pharmacology, Toxicology and Pharmaceutics（一般薬理学・毒物学・薬剤学）') ,'new_tag'] = '生命科学'


In [63]:
all_y_res['new_tag'].value_counts()

理学      1510
医学      1346
工学      1329
生命科学     601
その他      136
社会科学      84
Name: new_tag, dtype: int64

In [64]:
a = all_y_res.query('univ_name_x == "titech" or univ_name_x == "medical"')
a['univ_name_x'] = '東工大+医科歯科大'
all_y_res = pd.concat([all_y_res,a])

<ipython-input-64-509a954b6dec>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [65]:
all_y_res.head()

,Unnamed: 0,Scopus Author Ids,univ_name_x,start_year,end_year,All Science Journal Classification (ASJC) code,tag,univ_name_y,researcher_num,flag,new_tag
0,2,6602154752,titech,2012,2014,31,General Physics and Astronomy（一般物理・天文学）,osaka,1,医学以外,理学
1,3,36040908600,titech,2012,2014,25,General Materials Science（一般材料科学）,osaka,1,医学以外,工学
9,9,7202652119,titech,2012,2014,31,General Physics and Astronomy（一般物理・天文学）,osaka,1,医学以外,理学
12,11,7102848390,titech,2012,2014,16,General Chemistry（一般化学）,osaka,1,医学以外,理学
13,14,7102758574,titech,2012,2014,16,General Chemistry（一般化学）,osaka,1,医学以外,理学


In [66]:
re = all_y_res
re = re.groupby(['univ_name_x','tag']).sum().reset_index()
a = re.groupby(['univ_name_x'],as_index=False).sum()
b = a[['univ_name_x','researcher_num']]
c = pd.merge(re,b,how='outer',on='univ_name_x')
c['分野割合'] = c['researcher_num_x']/c['researcher_num_y']
d = c

In [67]:
#新しいタグ付きのデータフレーム(工学、理学、、)
re2 = all_y_res
re2 = re2.groupby(['univ_name_x','new_tag']).sum().reset_index()
a = re2.groupby(['univ_name_x'],as_index=False).sum()
b = a[['univ_name_x','researcher_num']]
c = pd.merge(re2,b,how='outer',on='univ_name_x')
c['分野割合'] = c['researcher_num_x']/c['researcher_num_y']
df_newtag = c

In [68]:
c.head()

,univ_name_x,new_tag,Unnamed: 0,Scopus Author Ids,start_year,end_year,All Science Journal Classification (ASJC) code,researcher_num_x,researcher_num_y,分野割合
0,Imperial,その他,29306,608912692082,36232,36268,279,18,1054,0.017078
1,Imperial,医学,744276,14148756335586,988546,989528,13115,491,1054,0.465844
2,Imperial,工学,507355,6421716567942,434957,435389,4661,216,1054,0.204934
3,Imperial,理学,448621,6105634488994,426899,427323,5121,212,1054,0.201139
4,Imperial,生命科学,145455,2947606245941,211446,211656,1874,105,1054,0.099620


In [69]:
all_y_res.query('univ_name_x == "titech"')['tag'].value_counts()

General Engineering（一般工学）                                        54
General Chemistry（一般化学）                                          34
General Physics and Astronomy（一般物理・天文学）                          33
General Materials Science（一般材料科学）                                22
General Computer Science（一般コンピューター科学）                            16
General Energy（一般エネルギー）                                           7
General Earth and Planetary Sciences（一般地球惑星科学）                    6
Biochemistry, Genetics and Molecular Biology（一般生化学・遺伝学・分子生物学）     6
General Environmental Science（一般環境科学）                             3
General Mathematics（一般数学）                                         1
General Medicine（一般医学）                                            1
Name: tag, dtype: int64

In [70]:
all_y_res['univ_name_x'].value_counts()

MIT          1246
Imperial     1054
UT            815
tohoku        475
kyoto         469
osaka         405
nagoya        285
東工大+医科歯科大     257
titech        183
medical        74
Name: univ_name_x, dtype: int64

In [71]:

d.loc[(d['tag'] != 'General Engineering（一般工学）')&(d['tag'] != 'General Chemistry（一般化学）')&
      (d['tag'] != 'General Physics and Astronomy（一般物理・天文学）')&(d['tag'] != 'General Materials Science（一般材料科学）')&
      (d['tag'] != 'General Computer Science（一般コンピューター科学）'),'tag'] = 'その他'

d.rename(columns={'univ_name_x':'大学名'},inplace=True)



In [72]:
fig = px.bar(df_newtag.query('univ_name_x == "titech"'),y='分野割合',x='new_tag',color='new_tag')
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.show()

In [73]:
two_yaxis_fig = make_subplots(specs=[[{"secondary_y": True}]]) 

two_yaxis_fig.add_trace(go.Bar(x = df_newtag.query('univ_name_x == "titech"')['new_tag'],
                               y = df_newtag.query('univ_name_x == "titech"')['分野割合']*100,
                               name='東工大'))

two_yaxis_fig.add_trace(go.Bar(x = df_newtag.query('univ_name_x == "MIT"')['new_tag'],
                               y = df_newtag.query('univ_name_x == "MIT"')['分野割合']*100,
                               name='MIT'))

two_yaxis_fig.add_trace(go.Bar(x = df_newtag.query('univ_name_x == "Imperial"')['new_tag'],
                               y = df_newtag.query('univ_name_x == "Imperial"')['分野割合']*100,
                               name='Imperial Colleage London'))

two_yaxis_fig.add_trace(go.Bar(x = df_newtag.query('univ_name_x == "UT"')['new_tag'],
                               y = df_newtag.query('univ_name_x == "UT"')['分野割合']*100,
                               name='東大'))

two_yaxis_fig.add_trace(go.Bar(x = df_newtag.query('univ_name_x == "kyoto"')['new_tag'],
                               y = df_newtag.query('univ_name_x == "kyoto"')['分野割合']*100,
                               name='京大'))

two_yaxis_fig.add_trace(go.Bar(x = df_newtag.query('univ_name_x == "osaka"')['new_tag'],
                               y = df_newtag.query('univ_name_x == "osaka"')['分野割合']*100,
                               name='大阪大'))

two_yaxis_fig.add_trace(go.Bar(x = df_newtag.query('univ_name_x == "tohoku"')['new_tag'],
                               y = df_newtag.query('univ_name_x == "tohoku"')['分野割合']*100,
                               name='東北大'))
two_yaxis_fig.add_trace(go.Bar(x = df_newtag.query('univ_name_x == "nagoya"')['new_tag'],
                               y = df_newtag.query('univ_name_x == "nagoya"')['分野割合']*100,
                               name='名古屋大'))

two_yaxis_fig.add_trace(go.Bar(x = df_newtag.query('univ_name_x == "medical"')['new_tag'],
                               y = df_newtag.query('univ_name_x == "medical"')['分野割合']*100,
                               name='医科歯科大'))

two_yaxis_fig.add_trace(go.Bar(x = df_newtag.query('univ_name_x == "東工大+医科歯科大"')['new_tag'],
                               y = df_newtag.query('univ_name_x == "東工大+医科歯科大"')['分野割合']*100,
                               name='東工大+医科歯科大'))


two_yaxis_fig.update_traces(
                  hovertemplate='%{x}: %{y:0.1f}',
                  texttemplate='%{y:0.1f}%',
                  textposition='outside')

two_yaxis_fig.update_layout(
    title = '2012~2021年にかけて条件を満たした研究者の分野割合',
    
    
)
two_yaxis_fig.show()

In [74]:
d1_order = {'東工大': 9, '東工大+医科歯科大':8,'MIT': 7,'Imperial College London': 6, '東大': 5,'京大':4,'阪大':3,'東北大':2,'名古屋大':1, '東京医科歯科大':0}
df_newtag.loc[df_newtag['univ_name_x'] == 'titech','univ_name_x'] = '東工大'
df_newtag.loc[df_newtag['univ_name_x'] == 'Imperial','univ_name_x'] = 'Imperial College London'
df_newtag.loc[df_newtag['univ_name_x'] == 'UT','univ_name_x'] = '東大'
df_newtag.loc[df_newtag['univ_name_x'] == 'kyoto','univ_name_x'] = '京大'
df_newtag.loc[df_newtag['univ_name_x'] == 'osaka','univ_name_x'] = '阪大'
df_newtag.loc[df_newtag['univ_name_x'] == 'tohoku','univ_name_x'] = '東北大'
df_newtag.loc[df_newtag['univ_name_x'] == 'nagoya','univ_name_x'] = '名古屋大'
df_newtag.loc[df_newtag['univ_name_x'] == 'medical','univ_name_x'] = '東京医科歯科大'
df_newtag.loc[df_newtag['univ_name_x'] == 'titech+medical','univ_name_x'] = '東工大+医科歯科大'

df_newtag['order'] = df_newtag['univ_name_x'].map(d1_order)
df_newtag = df_newtag.sort_values('order')

In [75]:
df_newtag.rename(columns={'univ_name_x' : '大学名', 'new_tag' : '分野'},inplace=True)

In [76]:
df_newtag.head()

,大学名,分野,Unnamed: 0,Scopus Author Ids,start_year,end_year,All Science Journal Classification (ASJC) code,researcher_num_x,researcher_num_y,分野割合,order
26,東京医科歯科大,理学,1120,56941318221,8062,8070,61,4,74,0.054054,0
27,東京医科歯科大,生命科学,1859,79026570564,8058,8066,77,4,74,0.054054,0
25,東京医科歯科大,工学,1926,216179686436,12087,12099,141,6,74,0.081081,0
24,東京医科歯科大,医学,15966,1527326914803,120908,121028,1651,60,74,0.810811,0
31,名古屋大,理学,71659,2479054782631,169175,169343,2345,84,285,0.294737,1


In [77]:
import math
df_newtag['分野割合(%)'] = df_newtag['分野割合']*100


In [78]:
df_newtag.head()

,大学名,分野,Unnamed: 0,Scopus Author Ids,start_year,end_year,All Science Journal Classification (ASJC) code,researcher_num_x,researcher_num_y,分野割合,order,分野割合(%)
26,東京医科歯科大,理学,1120,56941318221,8062,8070,61,4,74,0.054054,0,5.405405
27,東京医科歯科大,生命科学,1859,79026570564,8058,8066,77,4,74,0.054054,0,5.405405
25,東京医科歯科大,工学,1926,216179686436,12087,12099,141,6,74,0.081081,0,8.108108
24,東京医科歯科大,医学,15966,1527326914803,120908,121028,1651,60,74,0.810811,0,81.081081
31,名古屋大,理学,71659,2479054782631,169175,169343,2345,84,285,0.294737,1,29.473684


In [79]:
for u,t,p in zip(df_newtag['大学名'],df_newtag['分野'],df_newtag['分野割合(%)']):
  print(f'{u}の{t}分野の優秀な研究者は{p}%です')

東京医科歯科大の理学分野の優秀な研究者は5.405405405405405%です
東京医科歯科大の生命科学分野の優秀な研究者は5.405405405405405%です
東京医科歯科大の工学分野の優秀な研究者は8.108108108108109%です
東京医科歯科大の医学分野の優秀な研究者は81.08108108108108%です
名古屋大の理学分野の優秀な研究者は29.47368421052631%です
名古屋大の工学分野の優秀な研究者は25.964912280701753%です
名古屋大の医学分野の優秀な研究者は38.59649122807017%です
名古屋大のその他分野の優秀な研究者は1.0526315789473684%です
名古屋大の生命科学分野の優秀な研究者は3.8596491228070176%です
名古屋大の社会科学分野の優秀な研究者は1.0526315789473684%です
東北大の社会科学分野の優秀な研究者は0.21052631578947367%です
東北大の生命科学分野の優秀な研究者は6.526315789473684%です
東北大の理学分野の優秀な研究者は29.68421052631579%です
東北大の工学分野の優秀な研究者は36.63157894736842%です
東北大の医学分野の優秀な研究者は26.736842105263158%です
東北大のその他分野の優秀な研究者は0.21052631578947367%です
阪大の工学分野の優秀な研究者は28.641975308641975%です
阪大の医学分野の優秀な研究者は32.34567901234568%です
阪大の生命科学分野の優秀な研究者は6.91358024691358%です
阪大のその他分野の優秀な研究者は1.728395061728395%です
阪大の社会科学分野の優秀な研究者は0.49382716049382713%です
阪大の理学分野の優秀な研究者は29.876543209876544%です
京大の社会科学分野の優秀な研究者は2.345415778251599%です
京大の理学分野の優秀な研究者は39.01918976545842%です
京大の工学分野の優秀な研究者は18.336886993603414%です
京大の医学分野の優秀な研究者は23.667377398720

## 可視化
[全体の傾向]
- 条件を満たす研究者はほとんどの大学で工学と理学で5割以上を占める


[日本の大学の傾向]
- 東工大は単体だと理学と工学に条件を満たす研究者が偏っていた
- 東工大と医科歯科大が合併していたと仮定すると、条件を満たす研究者の割合は大阪大学や東北大と似てくる
- 東大では、理学分野において条件を満たす研究者の割合が非常に高く、工学分野と医学分野でも比較的高い傾向がある。
- 京都大学も東大と同様に理学分野において条件を満たす研究者の割合が非常に高く、生命科学分野と工学分野でも比較的高い傾向がある。
- 名古屋大学では、医学分野で条件を満たす研究者の割合が高い。
- 東北大学は工学分野と理学分野が強い
- 阪大では、医学分野と理学分野において条件を満たす研究者の割合が高い。


In [80]:
fig = px.bar(df_newtag, y='大学名', x='分野割合(%)', color='分野', orientation='h', text=[f'{val/sum(df_newtag["分野割合(%)"])*1000:.1f}%' for val in df_newtag["分野割合(%)"]])
fig.update_traces(textposition='outside')
fig.show()

# ③条件を満たす研究者の対象人数推移

## 集計

In [81]:
all_y_res['new_tag'].value_counts()

理学      1591
工学      1434
医学      1407
生命科学     611
その他      136
社会科学      84
Name: new_tag, dtype: int64

In [82]:
tag_all['研究者数'] = 1
tag_all['year'] = tag_all['start_year']
#tag_all.rename(columns={'start_year':'year'})
tag_all = tag_all.replace({'year': {2012: '2012~2014',2013:'2013~2015',2014:'2014~2016',2015:'2015~2017',2016:'2016~2018',2017:'2017~2019',2018:'2018~2020',2019:'2019~2021'}})
tag_all.rename(columns={'univ_name_x':'大学名'},inplace=True)
b = tag_all.groupby(['大学名','year','end_year'],as_index=False).sum()

line_tr = go.Scatter(x=b.query('大学名 == "titech"')['year'], y=b.query('大学名 == "titech"')['研究者数'], name='東工大')
line_tr1 = go.Scatter(x=b.query('大学名 == "MIT"')['year'], y=b.query('大学名 == "MIT"')['研究者数'], name='MIT')
line_tr2 = go.Scatter(x=b.query('大学名 == "Imperial"')['year'], y=b.query('大学名 == "Imperial"')['研究者数'], name='Imperial College London')
line_tr3 = go.Scatter(x=b.query('大学名 == "UT"')['year'], y=b.query('大学名 == "UT"')['研究者数'], name='東大')
line_tr4 = go.Scatter(x=b.query('大学名 == "kyoto"')['year'], y=b.query('大学名 == "kyoto"')['研究者数'], name='京大')
line_tr5 = go.Scatter(x=b.query('大学名 == "osaka"')['year'], y=b.query('大学名 == "osaka"')['研究者数'], name='大阪大')
line_tr6 = go.Scatter(x=b.query('大学名 == "tohoku"')['year'], y=b.query('大学名 == "tohoku"')['研究者数'], name='東北大')
line_tr7 = go.Scatter(x=b.query('大学名 == "nagoya"')['year'], y=b.query('大学名 == "nagoya"')['研究者数'], name='名古屋大')
line_tr8 = go.Scatter(x=b.query('大学名 == "medical"')['year'], y=b.query('大学名 == "medical"')['研究者数'], name='東京医科歯科大学')
line_tr9 = go.Scatter(x=b.query('大学名 == "東工大+医科歯科大"')['year'], y=b.query('大学名 == "東工大+医科歯科大"')['研究者数'], name='東工大+医科歯科大学')

subplots_fig = make_subplots(rows=1, cols=1)
subplots_fig.add_trace(line_tr, row=1, col=1)
subplots_fig.add_trace(line_tr1, row=1, col=1)
subplots_fig.add_trace(line_tr2, row=1, col=1)
subplots_fig.add_trace(line_tr3, row=1, col=1)
subplots_fig.add_trace(line_tr4, row=1, col=1)
subplots_fig.add_trace(line_tr5, row=1, col=1)
subplots_fig.add_trace(line_tr6, row=1, col=1)
subplots_fig.add_trace(line_tr7, row=1, col=1)
subplots_fig.add_trace(line_tr8, row=1, col=1)
subplots_fig.add_trace(line_tr9, row=1, col=1)

subplots_fig.update_layout(title = '条件を満たす研究者の推移')



## 全体の推移

### 全体の傾向
- 東工大は微減傾向
- 東工大と医科歯科大を合算した場合、名古屋大学や阪大に迫る
- Imperial College Londonが近年MITを超えた
- 東北大が京大を上回るなどの増加傾向がある
- 阪大と名大の差が縮んでいる



In [83]:
subplots_fig.show()

## 集計

In [84]:
#医学無しの場合の推移
not_med = tag_all.query('new_tag != "医学"')
a = not_med.groupby(['大学名','year','end_year'],as_index=False).sum()

line_trace = go.Scatter(x=a.query('大学名 == "titech"')['year'], y=a.query('大学名 == "titech"')['研究者数'], name='東工大')
line_trace1 = go.Scatter(x=a.query('大学名 == "MIT"')['year'], y=a.query('大学名 == "MIT"')['研究者数'], name='MIT')
line_trace2 = go.Scatter(x=a.query('大学名 == "Imperial"')['year'], y=a.query('大学名 == "Imperial"')['研究者数'], name='Imperial College London')
line_trace3 = go.Scatter(x=a.query('大学名 == "UT"')['year'], y=a.query('大学名 == "UT"')['研究者数'], name='東大')
line_trace4 = go.Scatter(x=a.query('大学名 == "kyoto"')['year'], y=a.query('大学名 == "kyoto"')['研究者数'], name='京大')
line_trace5 = go.Scatter(x=a.query('大学名 == "osaka"')['year'], y=a.query('大学名 == "osaka"')['研究者数'], name='大阪大')
line_trace6 = go.Scatter(x=a.query('大学名 == "tohoku"')['year'], y=a.query('大学名 == "tohoku"')['研究者数'], name='東北大')
line_trace7 = go.Scatter(x=a.query('大学名 == "nagoya"')['year'], y=a.query('大学名 == "nagoya"')['研究者数'], name='名古屋大')
line_trace8 = go.Scatter(x=a.query('大学名 == "medical"')['year'], y=a.query('大学名 == "medical"')['研究者数'], name='東京医科歯科大学')
line_trace9 = go.Scatter(x=a.query('大学名 == "東工大+医科歯科大"')['year'], y=a.query('大学名 == "東工大+医科歯科大"')['研究者数'], name='東工大+医科歯科大学')

subplots_fig = make_subplots(rows=1, cols=1)
subplots_fig.add_trace(line_trace, row=1, col=1)
subplots_fig.add_trace(line_trace1, row=1, col=1)
subplots_fig.add_trace(line_trace2, row=1, col=1)
subplots_fig.add_trace(line_trace3, row=1, col=1)
subplots_fig.add_trace(line_trace4, row=1, col=1)
subplots_fig.add_trace(line_trace5, row=1, col=1)
subplots_fig.add_trace(line_trace6, row=1, col=1)
subplots_fig.add_trace(line_trace7, row=1, col=1)
subplots_fig.add_trace(line_trace8, row=1, col=1)
subplots_fig.add_trace(line_trace9, row=1, col=1)

subplots_fig.update_layout(title='医学分野を除いた場合の条件を満たす研究者の推移')



## 医学分野を除いた場合の推移

### 全体の傾向
- 東北大と京都大学がほとんど同等になっている

### 東工大と日本の大学の比較
- 条件を満たす研究者の数は大阪大学や名古屋大学とほとんど変わらない


In [85]:
subplots_fig.show()

## 集計

In [86]:
#理学の推移
rigak = tag_all.query('new_tag == "理学"')
a = rigak.groupby(['大学名','year','end_year'],as_index=False).sum()

line_trace = go.Scatter(x=a.query('大学名 == "titech"')['year'], y=a.query('大学名 == "titech"')['研究者数'], name='東工大')
line_trace1 = go.Scatter(x=a.query('大学名 == "MIT"')['year'], y=a.query('大学名 == "MIT"')['研究者数'], name='MIT')
line_trace2 = go.Scatter(x=a.query('大学名 == "Imperial"')['year'], y=a.query('大学名 == "Imperial"')['研究者数'], name='Imperial College London')
line_trace3 = go.Scatter(x=a.query('大学名 == "UT"')['year'], y=a.query('大学名 == "UT"')['研究者数'], name='東大')
line_trace4 = go.Scatter(x=a.query('大学名 == "kyoto"')['year'], y=a.query('大学名 == "kyoto"')['研究者数'], name='京大')
line_trace5 = go.Scatter(x=a.query('大学名 == "osaka"')['year'], y=a.query('大学名 == "osaka"')['研究者数'], name='大阪大')
line_trace6 = go.Scatter(x=a.query('大学名 == "tohoku"')['year'], y=a.query('大学名 == "tohoku"')['研究者数'], name='東北大')
line_trace7 = go.Scatter(x=a.query('大学名 == "nagoya"')['year'], y=a.query('大学名 == "nagoya"')['研究者数'], name='名古屋大')
line_trace8 = go.Scatter(x=a.query('大学名 == "medical"')['year'], y=a.query('大学名 == "medical"')['研究者数'], name='東京医科歯科大学')
line_trace9 = go.Scatter(x=a.query('大学名 == "東工大+医科歯科大"')['year'], y=a.query('大学名 == "東工大+医科歯科大"')['研究者数'], name='東工大+医科歯科大学')

subplots_fig = make_subplots(rows=1, cols=1)
subplots_fig.add_trace(line_trace, row=1, col=1)
subplots_fig.add_trace(line_trace1, row=1, col=1)
subplots_fig.add_trace(line_trace2, row=1, col=1)
subplots_fig.add_trace(line_trace3, row=1, col=1)
subplots_fig.add_trace(line_trace4, row=1, col=1)
subplots_fig.add_trace(line_trace5, row=1, col=1)
subplots_fig.add_trace(line_trace6, row=1, col=1)
subplots_fig.add_trace(line_trace7, row=1, col=1)
subplots_fig.add_trace(line_trace8, row=1, col=1)
subplots_fig.add_trace(line_trace9, row=1, col=1)

subplots_fig.update_layout(title='理学分野の条件を満たす研究者の推移')



## 理学分野の推移


### 東工大と日本の大学の比較
- 東大、京大、東北大、名古屋大、大阪大、東工大の順になっている
- 東工大と医科歯科大を足すと阪大を越える
- 東大が抜きん出ている
- 東工大は名古屋大学,大阪大学とほぼ同等になっている


In [87]:
subplots_fig.show()

## 集計

In [88]:
#工学の推移
rigak = tag_all.query('new_tag == "工学"')
a = rigak.groupby(['大学名','year','end_year'],as_index=False).sum()

line_trace = go.Scatter(x=a.query('大学名 == "titech"')['year'], y=a.query('大学名 == "titech"')['研究者数'], name='東工大')
line_trace1 = go.Scatter(x=a.query('大学名 == "MIT"')['year'], y=a.query('大学名 == "MIT"')['研究者数'], name='MIT')
line_trace2 = go.Scatter(x=a.query('大学名 == "Imperial"')['year'], y=a.query('大学名 == "Imperial"')['研究者数'], name='Imperial College London')
line_trace3 = go.Scatter(x=a.query('大学名 == "UT"')['year'], y=a.query('大学名 == "UT"')['研究者数'], name='東大')
line_trace4 = go.Scatter(x=a.query('大学名 == "kyoto"')['year'], y=a.query('大学名 == "kyoto"')['研究者数'], name='京大')
line_trace5 = go.Scatter(x=a.query('大学名 == "osaka"')['year'], y=a.query('大学名 == "osaka"')['研究者数'], name='大阪大')
line_trace6 = go.Scatter(x=a.query('大学名 == "tohoku"')['year'], y=a.query('大学名 == "tohoku"')['研究者数'], name='東北大')
line_trace7 = go.Scatter(x=a.query('大学名 == "nagoya"')['year'], y=a.query('大学名 == "nagoya"')['研究者数'], name='名古屋大')
line_trace8 = go.Scatter(x=a.query('大学名 == "medical"')['year'], y=a.query('大学名 == "medical"')['研究者数'], name='東京医科歯科大学')
line_trace9 = go.Scatter(x=a.query('大学名 == "東工大+医科歯科大"')['year'], y=a.query('大学名 == "東工大+医科歯科大"')['研究者数'], name='東工大+医科歯科大学')

subplots_fig = make_subplots(rows=1, cols=1)
subplots_fig.add_trace(line_trace, row=1, col=1)
subplots_fig.add_trace(line_trace1, row=1, col=1)
subplots_fig.add_trace(line_trace2, row=1, col=1)
subplots_fig.add_trace(line_trace3, row=1, col=1)
subplots_fig.add_trace(line_trace4, row=1, col=1)
subplots_fig.add_trace(line_trace5, row=1, col=1)
subplots_fig.add_trace(line_trace6, row=1, col=1)
subplots_fig.add_trace(line_trace7, row=1, col=1)
subplots_fig.add_trace(line_trace8, row=1, col=1)
subplots_fig.add_trace(line_trace9, row=1, col=1)

subplots_fig.update_layout(title='工学分野の条件を満たす研究者の推移')



## 工学分野の推移


### 東工大と日本の大学の比較
- 東大、東北大が2強となっている
- 他の大学（東工大、京大、大阪大、名古屋大）はほぼ同数

In [89]:
subplots_fig.show()

## 集計

In [90]:
#生命科学の推移
rigak = tag_all.query('new_tag == "生命科学"')
a = rigak.groupby(['大学名','year','end_year'],as_index=False).sum()

line_trace = go.Scatter(x=a.query('大学名 == "titech"')['year'], y=a.query('大学名 == "titech"')['研究者数'], name='東工大')
line_trace1 = go.Scatter(x=a.query('大学名 == "MIT"')['year'], y=a.query('大学名 == "MIT"')['研究者数'], name='MIT')
line_trace2 = go.Scatter(x=a.query('大学名 == "Imperial"')['year'], y=a.query('大学名 == "Imperial"')['研究者数'], name='Imperial College London')
line_trace3 = go.Scatter(x=a.query('大学名 == "UT"')['year'], y=a.query('大学名 == "UT"')['研究者数'], name='東大')
line_trace4 = go.Scatter(x=a.query('大学名 == "kyoto"')['year'], y=a.query('大学名 == "kyoto"')['研究者数'], name='京大')
line_trace5 = go.Scatter(x=a.query('大学名 == "osaka"')['year'], y=a.query('大学名 == "osaka"')['研究者数'], name='大阪大')
line_trace6 = go.Scatter(x=a.query('大学名 == "tohoku"')['year'], y=a.query('大学名 == "tohoku"')['研究者数'], name='東北大')
line_trace7 = go.Scatter(x=a.query('大学名 == "nagoya"')['year'], y=a.query('大学名 == "nagoya"')['研究者数'], name='名古屋大')
line_trace8 = go.Scatter(x=a.query('大学名 == "medical"')['year'], y=a.query('大学名 == "medical"')['研究者数'], name='東京医科歯科大学')
line_trace9 = go.Scatter(x=a.query('大学名 == "東工大+医科歯科大"')['year'], y=a.query('大学名 == "東工大+医科歯科大"')['研究者数'], name='東工大+医科歯科大学')

subplots_fig = make_subplots(rows=1, cols=1)
subplots_fig.add_trace(line_trace, row=1, col=1)
subplots_fig.add_trace(line_trace1, row=1, col=1)
subplots_fig.add_trace(line_trace2, row=1, col=1)
subplots_fig.add_trace(line_trace3, row=1, col=1)
subplots_fig.add_trace(line_trace4, row=1, col=1)
subplots_fig.add_trace(line_trace5, row=1, col=1)
subplots_fig.add_trace(line_trace6, row=1, col=1)
subplots_fig.add_trace(line_trace7, row=1, col=1)
subplots_fig.add_trace(line_trace8, row=1, col=1)
subplots_fig.add_trace(line_trace9, row=1, col=1)

subplots_fig.update_layout(title='生命科学分野の条件を満たす研究者の推移')



## 生命科学分野の推移
- 理学と同じで東工大と医科歯科大を合算すると
- 全体的に少ない

In [91]:
subplots_fig.show()

# ④流動性可視化

- 研究力の高い大学は優秀な研究者が出入りしているのではないかという仮説
- 条件を満たす優秀な研究者の出入りを可視化することで、特徴を探る




## 集計

In [92]:
tag_all.drop('Unnamed: 0',axis=1,inplace = True)

In [93]:
tag_all.head()

,Scopus Author Ids,大学名,start_year,end_year,All Science Journal Classification (ASJC) code,tag,univ_name_y,new_tag,研究者数,year
0,6602154752,titech,2012,2014,31,General Physics and Astronomy（一般物理・天文学）,osaka,理学,1,2012~2014
1,36040908600,titech,2012,2014,25,General Materials Science（一般材料科学）,osaka,工学,1,2012~2014
2,36040908600,titech,2013,2015,25,General Materials Science（一般材料科学）,osaka,工学,1,2013~2015
3,36040908600,titech,2014,2016,25,General Materials Science（一般材料科学）,osaka,工学,1,2014~2016
4,36040908600,titech,2015,2017,25,General Materials Science（一般材料科学）,osaka,工学,1,2015~2017


In [94]:
tag_all.drop('univ_name_y',axis=1,inplace = True)

In [95]:
#研究者の流入など(大学別)
def make_df(univ_name):
  dff = pd.DataFrame()
  dff['years'] = 2012
  dff['前年と共通'] = 0
  dff['流入'] = 0
  dff['流出'] = 0
  year_ = ['2012~2014から2013~2015','2013~2015から2014~2016','2014~2016から2015~2017',
          '2015~2017から2016~2018','2016~2018から2017~2019','2018~2020から2019~2021']
  com_l = []
  out_l = []
  in_l = []
  for i in range(2013,2019):
    if i == 2019:
      break
    #print(i)
    titech_1 = tag_all[(tag_all['大学名'] == univ_name) & (tag_all['start_year'] == i)]
    titech_2 = tag_all[(tag_all['大学名'] == univ_name) & (tag_all['start_year'] == i+1)]
    f1to2 = pd.merge(titech_1,titech_2, on='Scopus Author Ids', how ="outer", indicator=True)
    #移動なしの研究者(共通)
    comm = f1to2.query('_merge == "both"').shape[0]
    com_l.append(comm)
    #流出した研究者数
    out_ = f1to2.query('_merge == "left_only"').shape[0]
    out_l.append(out_)
    #流入した研究者数
    in_ = f1to2.query('_merge == "right_only"').shape[0]
    in_l.append(in_)

  dff['years'] = year_
  dff['前年と共通'] = com_l
  dff['流出'] = out_l
  dff['流入'] = in_l
  return dff


In [96]:
a = make_df('titech')

In [97]:
#研究者の流入など(大学別)
def make_df2(univ_name,tag):
  dff = pd.DataFrame()
  dff['years'] = 2012
  dff['前年と共通'] = 0
  dff['流入'] = 0
  dff['流出'] = 0
  year_ = ['2012~2014から2013~2015','2013~2015から2014~2016','2014~2016から2015~2017',
          '2015~2017から2016~2018','2016~2018から2017~2019','2018~2020から2019~2021']
  com_l = []
  out_l = []
  in_l = []
  for i in range(2013,2019):
    if i == 2019:
      break
    #print(i)
    titech_1 = tag_all[(tag_all['大学名'] == univ_name) & (tag_all['start_year'] == i) & (tag_all['new_tag'] == tag)]
    titech_2 = tag_all[(tag_all['大学名'] == univ_name) & (tag_all['start_year'] == i+1) &  (tag_all['new_tag'] == tag)]
    f1to2 = pd.merge(titech_1,titech_2, on='Scopus Author Ids', how ="outer", indicator=True)
    #移動なしの研究者(共通)
    comm = f1to2.query('_merge == "both"').shape[0]
    com_l.append(comm)
    #流出した研究者数
    out_ = f1to2.query('_merge == "left_only"').shape[0]
    out_l.append(out_)
    #流入した研究者数
    in_ = f1to2.query('_merge == "right_only"').shape[0]
    in_l.append(in_)

  dff['years'] = year_
  dff['前年と共通'] = com_l
  dff['流出'] = out_l
  dff['流入'] = in_l
  return dff


In [98]:
def make_df3(univ_name):
  dff = pd.DataFrame()
  #dff['years'] = 2012
  dff['2012~2014から2019~2021年まで在籍'] = 0
  dff['流入'] = 0
  dff['流出'] = 0
  #year_ = ['2012~2014から2019~2021']
  com_l = []
  out_l = []
  in_l = []
  titech_1 = tag_all[(tag_all['大学名'] == univ_name) & (tag_all['start_year'] == 2012)]
  titech_2 = tag_all[(tag_all['大学名'] == univ_name) & (tag_all['start_year'] == 2019)]
  f1to2 = pd.merge(titech_1,titech_2, on='Scopus Author Ids', how ="outer", indicator=True)
  #移動なしの研究者(共通)
  comm = f1to2.query('_merge == "both"').shape[0]
  com_l.append(comm)
  #流出した研究者数
  out_ = f1to2.query('_merge == "left_only"').shape[0]
  out_l.append(out_)
  #流入した研究者数
  in_ = f1to2.query('_merge == "right_only"').shape[0]
  in_l.append(in_)
  #dff['years'] = year_
  dff['2012~2014から2019~2021年まで在籍'] = com_l
  dff['流出'] = out_l
  dff['流入'] = in_l
  return dff

## 2012 ~ 2014と2019 ~ 2021年の条件を満たす研究者の移動

In [99]:
dff = make_df3('titech')
two_yaxis_fig = make_subplots(specs=[[{"secondary_y": True}]]) 
two_yaxis_fig.add_trace(go.Bar(x = dff.T.index,
                               y = dff.T[0],
                               name='東工大'))
dff = make_df3('MIT')
two_yaxis_fig.add_trace(go.Bar(x = dff.T.index,
                               y = dff.T[0],
                               name='MIT'))
dff = make_df3('Imperial')
two_yaxis_fig.add_trace(go.Bar(x = dff.T.index,
                               y = dff.T[0],
                               name='Imperial College London'))
dff = make_df3('UT')
two_yaxis_fig.add_trace(go.Bar(x = dff.T.index,
                               y = dff.T[0],
                               name='東大'))
dff = make_df3('kyoto')
two_yaxis_fig.add_trace(go.Bar(x = dff.T.index,
                               y = dff.T[0],
                               name='京大'))
dff = make_df3('tohoku')
two_yaxis_fig.add_trace(go.Bar(x = dff.T.index,
                               y = dff.T[0],
                               name='東北大'))
dff = make_df3('osaka')
two_yaxis_fig.add_trace(go.Bar(x = dff.T.index,
                               y = dff.T[0],
                               name='大阪大'))
dff = make_df3('nagoya')
two_yaxis_fig.add_trace(go.Bar(x = dff.T.index,
                               y = dff.T[0],
                               name='名古屋大'))
two_yaxis_fig.update_traces(
    selector= 'bar',
    hovertemplate='%{x}: %{y:1f}',
    texttemplate='%{y:1f}',
    textposition='outside'
)

two_yaxis_fig.update_layout(
    title = '2012 ~ 2014と2019 ~ 2021年の条件を満たす研究者の移動'
)
two_yaxis_fig.show()

## 可視化
- MITの流出入が多い
- Imperial College Londonと東大はそのまま在籍している条件を満たす研究者が多い
- 東工大は規模から比較して条件を満たす研究者の移動は多い

In [100]:
two_yaxis_fig.show()
y_axis.show()

# まとめ
- 東工大の条件を満たす研究者は概ね横ばいもしくは微減している
- 分野ごとに見た場合、東工大は工学と理学分野では阪大、名古屋大と同程度（大学の規模から考えると割合は高い）
- 東工大と医科歯科大学が合併した場合、条件を満たす研究者の分野割合は大阪大学や東北大学と近くなる
- 日本の主要大学で比較した場合、①東大,②京大,東北大,③大阪大、名古屋大、東工大の３つのグループに分けられる
- 研究力が高いと考えられる大学（MIT,Imperial College London,東大）は該当する研究者の移動が多い（MIT）もしくは在籍し続ける研究者が多い（東大,Imperial）の二極化しているように思われる→国別の要因？分野ごとに特徴があったりするのか？
パフォーマンスが低下した場合にカウントされなくなるのでそこを注意する